In [ ]:
# setups
import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

sess = Session()

role = get_execution_role()

pytorch_model = PyTorchModel(model_data="s3://modelartifactdata/my_model.tar.gz",
                             source_dir='code',
                             role=role,
                             entry_point='bethInference.py',
                             py_version="py39",
                             framework_version="1.13.1")

predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge', 
                                 initial_instance_count=1,
                                 serializer=JSONSerializer(),
                                 deserializer=JSONDeserializer(),
                                 endpoint_name='BethEndpoint')


----

In [ ]:
res = predictor.predict("s3://training-and-validation/sample_single_safe.csv")

In [ ]:
print(res)

In [ ]:
predictor.delete_endpoint()

In [56]:
import csv
import re
s3 = boto3.client('s3')
obj = s3.get_object(Bucket='training-and-validation', Key='labelled_testing_data.csv')
data = obj['Body'].read().decode('utf-8').splitlines()
result = []
for eachRecord in data[1000:2000]:
    predVal = predictor.predict(str(eachRecord))
    test = int(re.search(r'\d+', predVal).group())
    while test > 100:
        predVal = predictor.predict(str(eachRecord))
        test = int(re.search(r'\d+', predVal).group())
                                    
    temp = [eachRecord[len(eachRecord)-3],eachRecord[len(eachRecord)-1], predVal]
    result.append(temp)

In [51]:
# opening the csv file in 'w+' mode
file = open('result.csv', 'w+', newline ='')
 
# writing the data into the file
with file:   
    write = csv.writer(file)
    write.writerow(result[0])
    
file.close()